## 0. Introduction

Ce projet est réalisé par Zakarya El Mimouni et Yassine Boukhateb étudiants en deuxième année du cycle ingénieur de l'ENSAE Paris dans le cadre du cours de Python pour la DataScience.

**Importation des modules et packages**

In [1]:
import requests
import json
import time
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

## I. Récupération des données

On récupère les données de 10 000 films grâce à l'API TMDB

In [6]:
# Votre clé d'API TMDb
api_key = '28c6630049f5d468217e4f34963c03a6'

# URL de base de l'API TMDb pour les films populaires
base_url_popular = 'https://api.themoviedb.org/3/movie/popular'

# Nombre total de films à récupérer
total_movies = 20

# Nombre de films à récupérer par requête
movies_per_request = 20 

# Champs de base pour les films
fields = 'id'##,title,original_title,overview,release_date,popularity,vote_average,vote_count,poster_path,runtime,genres,production_companies,production_countries,spoken_languages'

# Liste pour stocker tous les films avec leurs détails
all_movies_details = []

# Nombre de requêtes nécessaires pour obtenir le nombre total de films
num_requests = total_movies // movies_per_request

# Effectuer les requêtes pour récupérer les films
for page in range(1, num_requests + 1):
    params = {
        'api_key': api_key,
        'page': page,
        'fields': fields
    }

    response = requests.get(base_url_popular, params=params)

    if response.status_code == 200:
        movies_data = response.json()
        movies = movies_data.get('results', [])

        # Pour chaque film, récupérer les détails supplémentaires
        for movie in movies:
            movie_id = movie['id']
            movie_details_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&append_to_response=credits,keywords'

            movie_details_response = requests.get(movie_details_url)
            if movie_details_response.status_code == 200:
                movie_details = movie_details_response.json()
                all_movies_details.append(movie_details)
                print(f"Film {movie['title']} récupéré avec détails supplémentaires.")
                time.sleep(0.5)
            else:
                print(f"Erreur lors de la récupération des détails pour {movie['title']}. Statut : {movie_details_response.status_code}")

        print(f"Page {page} récupérée. Total de films récupérés : {len(all_movies_details)}")
        time.sleep(0.5)
    else:
        print(f"Erreur lors de la requête pour la page {page}. Statut : {response.status_code}")

    if len(all_movies_details) >= total_movies:
        break

# Enregistrer les données complètes dans un fichier JSON
with open('movies_data_complete.json', 'w', encoding='utf-8') as f:
    json.dump(all_movies_details, f, ensure_ascii=False, indent=4)

print("Récupération des films avec détails complets terminée. Les données ont été enregistrées dans movies_data_complete.json.")


Film Killers of the Flower Moon récupéré avec détails supplémentaires.
Film Robot Apocalypse récupéré avec détails supplémentaires.
Film Freelance récupéré avec détails supplémentaires.
Film Wonka récupéré avec détails supplémentaires.
Film Trolls Band Together récupéré avec détails supplémentaires.
Film Leave the World Behind récupéré avec détails supplémentaires.
Film Leo récupéré avec détails supplémentaires.
Film Good Boy récupéré avec détails supplémentaires.
Film A Man of Reason récupéré avec détails supplémentaires.
Film Oppenheimer récupéré avec détails supplémentaires.
Film Five Nights at Freddy's récupéré avec détails supplémentaires.
Film The Creator récupéré avec détails supplémentaires.
Film The Advent Calendar récupéré avec détails supplémentaires.
Film Rumble Through the Dark récupéré avec détails supplémentaires.
Film Fast X récupéré avec détails supplémentaires.
Film Fast Charlie récupéré avec détails supplémentaires.
Film Merry Little Batman récupéré avec détails supp

In [21]:
movies_data=pd.read_json('movies_data_complete.json')
selected_columns=['id','title','original_title','overview','release_date','adult','popularity','vote_average','vote_count','poster_path','runtime','genres','production_companies','production_countries','spoken_languages','credits','keywords']
movies_data=movies_data=movies_data[selected_columns]
movies_data.head()

,id,title,original_title,overview,release_date,adult,popularity,vote_average,vote_count,poster_path,runtime,genres,production_companies,production_countries,spoken_languages,credits,keywords
0,466420,Killers of the Flower Moon,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,2023-10-18,False,1323.647,7.617,1445,/dB6Krk806zeqd0YNp2ngQ9zXteH.jpg,206,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[{'id': 194232, 'logo_path': '/oE7H93u8sy5vvW5...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 1157, 'name': 'husband wi..."
1,854648,Robot Apocalypse,Robot Apocalypse,An expert hacker is targeted by a sentient AI ...,2021-07-26,False,692.968,6.048,62,/zpbgktIR7GkOS83PBAzLlzLSQ5W.jpg,87,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...","[{'id': 1311, 'logo_path': '/ic2bTizdzRLDVzAvN...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...",{'keywords': []}
2,897087,Freelance,Freelance,An ex-special forces operative takes a job to ...,2023-01-05,False,1243.624,6.535,317,/7Bd4EUOqQDKZXA6Od5gkfzRNb0.jpg,108,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...","[{'id': 89171, 'logo_path': '/c3ttVfx0itQzk2vO...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 6285, 'name': 'coup d'eta..."
3,787699,Wonka,Wonka,Willy Wonka – chock-full of ideas and determin...,2023-12-06,False,1119.142,7.317,278,/qhb1qOilapbapxWQn9jtRCMwXJF.jpg,117,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...","[{'id': 174, 'logo_path': '/IuAlhI9eVC9Z8UQWOI...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 715, 'name': 'chocolate'}..."
4,901362,Trolls Band Together,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...",2023-10-12,False,968.800,7.169,370,/bkpPTZUdq31UGDovmszsg2CchiI.jpg,92,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...","[{'id': 521, 'logo_path': '/kP7t6RwGz2AvvTkvnI...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 1, 'id': ...","{'keywords': [{'id': 3490, 'name': 'pop star'}..."


In [32]:
movies_data.isnull().sum()


id                      0
title                   0
original_title          0
overview                0
release_date            0
adult                   0
popularity              0
vote_average            0
vote_count              0
poster_path             0
runtime                 0
genres                  0
production_companies    0
production_countries    0
spoken_languages        0
credits                 0
keywords                0
dtype: int64

In [33]:
movies_data.dropna(axis=0,inplace=True)